Name: Saman Akhtar USC ID: 9944619932

**Imports**

In [5]:

import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [ ]:
print("Num GPUs Available: ", 
      len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [6]:

from google.colab import drive
drive.mount('/content/drive')
#drive.mount('../data/books')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
drive_folder = '/content/drive/My Drive/ML/homework-7-SamanUSC'

# **Generative Models for Text**


 ## **Download the following books from Project Gutenberg http://www.gutenberg.
## org/ebooks/author/355 in text format:**

###  LSTM: Train an LSTM to mimic Russell’s style and thoughts:

#### i. Concatenate your text files to create a corpus of Russell’s writings

In [8]:
def read_books(dir):
  print(" The current directory", dir)

  # Change the directory
  os.chdir(dir)
  corpus_lst = []

  # Iterate through all file
  for file in os.listdir():

      # Check whether file is in the text format or not
      if file.endswith(".txt"):
        file_path = f"{dir}/{file}"
   
      with open(file_path, encoding = 'ascii', errors = 'ignore') as corp:

        # Reading all the text in lower case
        corpus = corp.read().lower()
        corpus_lst.append(corpus)
      print(" Read the text of the book:", file, " of the length: ", len(corpus_lst[-1]))
  corpus_lst = sorted(corpus_lst, key=lambda x : len(x))
  return corpus_lst 
        

In [9]:
# Create corpus of Russell's writings
all_corpus = read_books(drive_folder + '/data/books/')

 The current directory /content/drive/My Drive/ML/homework-7-SamanUSC/data/books/
 Read the text of the book: TAMatter.txt  of the length:  766542
 Read the text of the book: THWP.txt  of the length:  2005566
 Read the text of the book: TPP.txt  of the length:  244306
 Read the text of the book: MLOE.txt  of the length:  412226
 Read the text of the book: OKEWFSMP.txt  of the length:  405741
 Read the text of the book: TAM.txt  of the length:  514652
 Read the text of the book: AIIMAT.txt  of the length:  746219


#### ii. Use a character-level representation for this model by using extended ASCII that has N = 256 characters. Each character will be encoded into a an integer using its ASCII code. Rescale the integers to the range [0, 1], because LSTM uses a sigmoid activation function. LSTM will receive the rescaled integers as its input

In [10]:
def getRepresentation(all_corpus):
  all_char = []
  for corpus in all_corpus:

      # Get 1 set of all characters in the corpus   
      all_char.extend(list(set(set(corpus))))
  all_char = sorted(list(set(all_char)))
  # rem_char= ['|' ,'#', '%' , '~', '$', '+','^']
  # all_char = set(all_char) - set(rem_char)
  charInt = dict((c, i) for i, c in enumerate(all_char))
  intChar = dict((i, c) for i, c in enumerate(all_char))
  return charInt, intChar, all_char

In [11]:
charInt, intChar, all_char = getRepresentation(all_corpus)
print(charInt)
print(intChar)

{'\n': 0, ' ': 1, '!': 2, '"': 3, '#': 4, '$': 5, '%': 6, '&': 7, "'": 8, '(': 9, ')': 10, '*': 11, '+': 12, ',': 13, '-': 14, '.': 15, '/': 16, '0': 17, '1': 18, '2': 19, '3': 20, '4': 21, '5': 22, '6': 23, '7': 24, '8': 25, '9': 26, ':': 27, ';': 28, '<': 29, '=': 30, '>': 31, '?': 32, '[': 33, '\\': 34, ']': 35, '^': 36, '_': 37, 'a': 38, 'b': 39, 'c': 40, 'd': 41, 'e': 42, 'f': 43, 'g': 44, 'h': 45, 'i': 46, 'j': 47, 'k': 48, 'l': 49, 'm': 50, 'n': 51, 'o': 52, 'p': 53, 'q': 54, 'r': 55, 's': 56, 't': 57, 'u': 58, 'v': 59, 'w': 60, 'x': 61, 'y': 62, 'z': 63, '{': 64, '|': 65, '}': 66, '~': 67}
{0: '\n', 1: ' ', 2: '!', 3: '"', 4: '#', 5: '$', 6: '%', 7: '&', 8: "'", 9: '(', 10: ')', 11: '*', 12: '+', 13: ',', 14: '-', 15: '.', 16: '/', 17: '0', 18: '1', 19: '2', 20: '3', 21: '4', 22: '5', 23: '6', 24: '7', 25: '8', 26: '9', 27: ':', 28: ';', 29: '<', 30: '=', 31: '>', 32: '?', 33: '[', 34: '\\', 35: ']', 36: '^', 37: '_', 38: 'a', 39: 'b', 40: 'c', 41: 'd', 42: 'e', 43: 'f', 44: 'g

#### iii) Choose a window size, iv. Inputs to the network will be the first W −1 = 99 characters of each sequence, and the output of the network will be the Wth character of the sequence. Basically, we are training the network to predict each character using the 99 characters that precede it. Slide the window in strides of S = 1 on the text.For example, if W = 5 and S = 1 and we want to train the network with the sequence ABRACADABRA, The first input to the network will be ABRA and the corresponding output will be C. The second input will be BRAC and the second output will be A, etc.


In [12]:
def windCorpus(corpus, charDict, wind_size ):
    input = []
    output = []
    for w in range(0, len(corpus) - wind_size + 1, 1):

        # Storing  W − 1 characters of each sequence as input
        seqIn = corpus[w : w + wind_size - 1]

        # storing W character of each sequence as output
        seqOut = corpus[w + wind_size - 1]
        input.append([charDict[c] for c in seqIn])
        output.append(charDict[seqOut])
    print(len(output))
    return input, output

In [13]:
def genInOut(corpus, charDict, wind_size = 100, n = 7):
  inSeq = []
  outChar = []
  for book in corpus[:n]:
    input, output = windCorpus(book, charDict, wind_size)
    inSeq.extend(input)
    outChar.extend(output)
  return inSeq, outChar

Choosing window size as 100

In [14]:
wind_size = 100
n = 5
inSeq, outChar = genInOut(all_corpus, charInt, wind_size, n)

244207
405642
412127
514553
746120


#### v. Note that the output has to be encoded using a one-hot encoding scheme with N = 256 (or less) elements. This means that the network reads integers, but outputs a vector of N = 256 (or less) elements. 

In [15]:
print(len(inSeq))
# Reshape
# input to LSTM layer should be in 3D shape (samples, time-steps, features)
lstm_in = np.reshape(inSeq, (len(inSeq), wind_size - 1, 1))
print(lstm_in.shape)

2322649
(2322649, 99, 1)


In [16]:
# Normalize
lstm_in = lstm_in / float(len(charInt))

In [17]:
lstm_out = np_utils.to_categorical(outChar)
print(lstm_out.shape)

(2322649, 68)


#### vi. Use a single hidden layer for the LSTM withN= 256 (or less) memory units.vii. Use a Softmax output layer to yield a probability prediction for each of thecharacters between 0 and . This is actually a character classification problemwithNclasses. Choose log loss (cross entropy) as the objective function forthe network .

In [18]:
# Build LSTM Sequential model
model = Sequential()

# Input to LSTM layer should be in 3D shape (samples, time-steps, features)
model.add(LSTM(256, input_shape=(lstm_in.shape[1], lstm_in.shape[2])))
model.add(Dropout(0.2))

# Add one LSTM layer.
model.add(Dense(lstm_out.shape[1], activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 256)               264192    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 68)                17476     
Total params: 281,668
Trainable params: 281,668
Non-trainable params: 0
_________________________________________________________________
None


#### viii. We do not use a test dataset. We are using the whole training dataset tolearn the probability of each character in a sequence. We are not seeking fora very accurate model. Instead we are interested in a generalization of thedataset that can mimic the gist of the text.ix. Choose a reasonable number of epochs4for training, considering your compu-tational power .x. Use model checkpointing to keep the network weights to determine each timean improvement in loss is observed at the end of the epoch. Find the best setof weights in terms of loss

In [19]:
# Configuring models learning process with .compile():
model.compile(loss='categorical_crossentropy', optimizer='adam')
filepath = drive_folder + "/LSTMweights/weights-improvement-{epoch:02d}-{loss:.2f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

Choosing epoch as 25 and batch size as 128

In [23]:
# fit the model
model.fit(lstm_in, lstm_out, epochs = 20, batch_size = 128, callbacks = callbacks_list)

Epoch 1/20
18146/18146 [==============================] - 264s 14ms/step - loss: 2.6697

Epoch 00001: loss improved from inf to 2.66970, saving model to /content/drive/My Drive/ML/homework-7-SamanUSC/LSTMweights/weights-improvement-01-2.67-bigger.hdf5
Epoch 2/20
18146/18146 [==============================] - 258s 14ms/step - loss: 2.3157

Epoch 00002: loss improved from 2.66970 to 2.31574, saving model to /content/drive/My Drive/ML/homework-7-SamanUSC/LSTMweights/weights-improvement-02-2.32-bigger.hdf5
Epoch 3/20
18146/18146 [==============================] - 258s 14ms/step - loss: 2.1182

Epoch 00003: loss improved from 2.31574 to 2.11822, saving model to /content/drive/My Drive/ML/homework-7-SamanUSC/LSTMweights/weights-improvement-03-2.12-bigger.hdf5
Epoch 4/20
18146/18146 [==============================] - 258s 14ms/step - loss: 2.1737

Epoch 00004: loss did not improve from 2.11822
Epoch 5/20
18146/18146 [==============================] - 258s 14ms/step - loss: 2.9647

Epoch 00005

#### xi Use the network with the best weights to generate 1000 characters, using thefollowing text as initialization of the network:There are those who take mental phenomena naively, just as theywould physical phenomena. This school of psychologists tends not toemphasize the object.

In [30]:
txt = 'There are those who take mental phenomena naively, just as they would physical phenomena. This school of psychologists tends not to emphasize the object.'

In [29]:
text_int = [charInt[c] for c in txt[-99:].lower()]

In [22]:

def generateNewTxt(txt , text_int, charInt, intChar, model, ch_len = 1000):
  for i in range(ch_len):

    # Reshape to 3D array
    seq = np.reshape(text_int, (1, len(text_int), 1))

    # Nomalize it
    seq = seq / float(len(charInt))

    # Predict the next character
    nxt_char = model.predict(seq, verbose=0)
    char_index = np.argmax(nxt_char)
    txt += intChar[char_index]

    # Generate new input sequence
    text_int.append(char_index)
    text_int = text_int[1:len(text_int)]
  return txt

In [24]:
new_txt = generateNewTxt(txt , text_int, charInt, intChar, model,  1000)

In [25]:
print(" The new text generated by LSTM is \n:", new_txt)

 The new text generated by LSTM is 
: There are those who take mental phenomena naively, just as they would physical phenomena. This school of psychologists tends not to emphasize the object. the semsetier of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense o

Fitting model with more epoc

In [26]:
model.fit(lstm_in, lstm_out, epochs = 10, batch_size = 128, callbacks = callbacks_list)

Epoch 1/10
18146/18146 [==============================] - 258s 14ms/step - loss: 1.7655

Epoch 00001: loss improved from 1.77872 to 1.76555, saving model to /content/drive/My Drive/ML/homework-7-SamanUSC/LSTMweights/weights-improvement-01-1.77-bigger.hdf5
Epoch 2/10
18146/18146 [==============================] - 257s 14ms/step - loss: 1.7530

Epoch 00002: loss improved from 1.76555 to 1.75303, saving model to /content/drive/My Drive/ML/homework-7-SamanUSC/LSTMweights/weights-improvement-02-1.75-bigger.hdf5
Epoch 3/10
18146/18146 [==============================] - 257s 14ms/step - loss: 1.7424

Epoch 00003: loss improved from 1.75303 to 1.74238, saving model to /content/drive/My Drive/ML/homework-7-SamanUSC/LSTMweights/weights-improvement-03-1.74-bigger.hdf5
Epoch 4/10
18146/18146 [==============================] - 257s 14ms/step - loss: 1.7322

Epoch 00004: loss improved from 1.74238 to 1.73216, saving model to /content/drive/My Drive/ML/homework-7-SamanUSC/LSTMweights/weights-improvem

In [33]:
newtxt = generateNewTxt(txt , text_int, charInt, intChar, model,  1000)

In [34]:
print(" The new text generated by LSTM is \n:", newtxt)

 The new text generated by LSTM is 
: There are those who take mental phenomena naively, just as they would physical phenomena. This school of psychologists tends not to emphasize the object.the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and the senset and

**Referrences**

https://www.geeksforgeeks.org/how-to-read-multiple-text-files-from-folder-in-python/ https://keras.io/guides/functional_api/ https://www.tensorflow.org/tutorials/keras/ https://stackabuse.com/solving-sequence-problems-with-lstm-in-keras https://machinelearningmastery.com/reshape-input-data-long-short-term-memory-networks-keras/

